# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [3]:
from trading_bot.agent import Agent

model_name = 'GOOG1d_50'
test_stock = 'data/GOOG_1d_test.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [4]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-30,1089.060059
1,2019-01-31,1116.369995
2,2019-02-01,1110.750000
3,2019-02-04,1132.800049
4,2019-02-05,1145.989990


### 3. Running Eval

In [5]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-07 11:48:22 justin-desktop-ubuntu root[33028] DEBUG switching to TensorFlow for CPU
2020-09-07 11:48:22 justin-desktop-ubuntu root[33028] DEBUG Buy at: $1089.06
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Sell at: $1116.37 | Position: +$27.31
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Buy at: $1231.54
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Sell at: $1205.50 | Position: -$26.04
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Buy at: $1092.50
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Sell at: $1111.42 | Position: +$18.92
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Buy at: $1177.60
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Sell at: $1192.85 | Position: +$15.25
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Buy at: $1241.39
2020-09-07 11:48:23 justin-desktop-ubuntu root[33028] DEBUG Sell at: $1225.09 | Position: -$16.30
2020-09-07 11:48:23 justin-deskto

Final decision:  HOLD  at  $1641.84


### 4. Visualize

In [6]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [7]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)